# Performance comparison

In [87]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import r2_score

In [141]:
df = pd.read_csv('data/laptop_test.csv')
df = df.dropna()

In [82]:
df.head()

,company,product,typename,inches,cpu,ram(GB),gpu,opsys,weight(kg),resolution,screentype,touchscreen,cpu(GHz),memory_1_storage_gb,memory_1_type,memory_2_storage_gb,memory_2_type,log_price
0,16,334,3,13.3,57,8,53,5,1.05,3,0,0,2.5,256.0,3,0.0,3,7.421776
1,7,417,3,13.3,57,8,53,5,1.11,3,0,0,2.5,256.0,3,0.0,3,7.046647
2,10,202,3,15.6,0,6,18,5,2.40,3,2,0,2.4,1000.0,1,0.0,3,6.212606
3,10,225,3,15.6,69,8,89,5,2.60,3,0,0,2.6,1000.0,1,0.0,3,6.801283
4,7,110,4,12.5,52,4,47,5,1.26,0,2,0,2.3,256.0,3,0.0,3,7.126087


In [45]:
target = "log_price"
numericFeatures = ["inches", "ram(GB)", "weight(kg)", "cpu(GHz)", "memory_1_storage_gb", "memory_2_storage_gb"]
catFeatures = ["company", "product", "typename", "cpu", "gpu", "opsys", "resolution", "screentype", "touchscreen", "memory_1_type", "memory_2_type"]

features = df.columns.tolist()
features.remove(target)

## Predictions on test data
Never seen dataest

### Random Forest

In [80]:
random_forest = pickle.load(open("models/random_forest.sav", 'rb'))
random_forest_pred = random_forest.predict(df[features])
random_forest_r2 = r2_score(random_forest_pred, df[target])
print("Random Forest\nR2: {:.4f} %".format(random_forest_r2*100))

Random Forest
R2: 93.0297 %


### XGBoost

In [86]:
xgb = pickle.load(open("models/xgboost.sav", 'rb'))
xgb_pred = xgb.predict(df[features])
xgb_r2 = r2_score(xgb_pred, df[target])
print("XGBoost\nR2: {:.4f} %".format(xgb_r2*100))

[00:54:27] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
XGBoost
R2: 91.3395 %


### Neural Network

In [155]:
from pickle import load
from keras.models import load_model

In [156]:
df_nn = df.copy()
scaler = load(open('scaler/scaler.pkl', 'rb'))

In [157]:
features = df.columns.tolist()
features.remove(target)

In [158]:
X = scaler.transform(df_nn[features])

In [159]:
df_Y = pd.DataFrame(df_nn[target])
df_pred = pd.DataFrame(neural_network.predict(X))

df_Y.reset_index(drop=True, inplace=True)
df_pred.reset_index(drop=True, inplace=True)

df_results = pd.concat([df_Y, df_pred], axis=1)
df_results.columns = ['Actual', 'Predicted']

df_results

4/4 [==============================] - 0s 3ms/step


,Actual,Predicted
0,7.421776,7.228240
1,7.046647,6.960146
2,6.212606,6.004714
3,6.801283,6.799929
4,7.126087,7.489755
...,...,...
121,7.002156,7.251554
122,6.475433,6.453342
123,7.229114,7.742352
124,6.549651,7.189648


In [167]:
#neural_network = pickle.load(open("models/neural_network.sav", 'rb'))
neural_network = load_model("models/neural_network.h5")
neural_network_pred = neural_network.predict(X)
neural_network_r2 = r2_score(neural_network_pred, df_nn[target])
print("Neural Network\nR2: {:.4f} %".format(neural_network_r2*100))

4/4 [==============================] - 0s 3ms/step
Neural Network
R2: 75.5084 %


## Summary

In [172]:
print("Random Forest\nR2: {0:.4f} %\n\nXGBoost\nR2: {1:.4f} %\n\nNeural Network\nR2: {2:.4f} %".format(random_forest_r2*100, xgb_r2*100, neural_network_r2*100))

Random Forest
R2: 93.0297 %

XGBoost
R2: 91.3395 %

Neural Network
R2: 75.5084 %
